## Imports

In [1]:
import io
import os.path
from scrapy import Selector
from datetime import datetime
from urllib.request import Request, urlopen

## Baixando HTML do site Fundamentus (Web Crawling)

In [46]:
# mudar o nome do papel para baixar novo HTML
papel = 'ITUB4'
url = 'https://fundamentus.com.br/detalhes.php?papel=%s' % papel

req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})

response = urlopen(req, timeout=20).read()
new_html= response.decode('latin-1')

#### Verificando data do último balanço

In [47]:
sel = Selector( text = new_html )

In [48]:
data_ult_balanco_new_html = sel.xpath("//table[2]//span[@class='txt']/text()").extract()[3]
data_ult_balanco_new_html = datetime.strptime(data_ult_balanco_new_html, '%d/%m/%Y').date()
data_ult_balanco_new_html = str(data_ult_balanco_new_html)
data_ult_balanco_new_html

'2020-12-31'

In [49]:
path = '/home/hugo/Documents/Repositorios_GitHub/Projetos/02-StockData/Web_Scraping/Fundamentus_Web/'
file_name = '%s_fundamentus_%s.txt' % (papel, str(data_ult_balanco_new_html))

if os.path.isfile(path+file_name):
    print('Arquivo já existe com balanço mais atual!')
else:
    print('Arquivo com balanço atual ainda não existe!')

Arquivo já existe com balanço mais atual!


## Salvando conteúdo do HTML (em bytes) em um arquivo local como txt

In [50]:
file_name = '%s_fundamentus_%s.txt' % (papel, data_ult_balanco_new_html)

with io.open(path+file_name, 'w', encoding='utf-8') as f:
    f.write(new_html)

## Lendo conteúdo do arquivo Local e armazenando em uma variável

In [51]:
with io.open(path+file_name, 'r', encoding='utf8') as f:
    html = f.read()

## Extrair e Separar Labels dos Dados do HTML (Web Scraping)
### Tabela 1 e 2: Dados da Empresa e última cotação

In [52]:
#!pip3 install scrapy

In [53]:
sel = Selector( text = html )

In [54]:
empresa_cotacao = sel.xpath("//table[1]//span[@class='txt']/text() | //table[1]//span[@class='txt']/a/text()").extract()
valores_data_balanco = sel.xpath("//table[2]//span[@class='txt']/text()").extract()

In [55]:
empresa_label = empresa_cotacao[0::2] + valores_data_balanco[0::2]
empresa_label

['Papel',
 'Cotação',
 'Tipo',
 'Data últ cot',
 'Empresa',
 'Min 52 sem',
 'Setor',
 'Max 52 sem',
 'Subsetor',
 'Vol $ méd (2m)',
 'Valor de mercado',
 'Últ balanço processado',
 'Valor da firma',
 'Nro. Ações']

In [56]:
empresa_dados = empresa_cotacao[1::2] + valores_data_balanco[1::2]
empresa_dados

['ITUB4',
 '26,08',
 'PN N1',
 '25/02/2021',
 'ITAUUNIBANCO PN N1',
 '20,36',
 'Intermediários Financeiros',
 '32,78',
 'Bancos',
 '1.160.530.000',
 '255.692.000.000',
 '31/12/2020',
 '-',
 '9.804.140.000']

### Tabela 3.1: Oscilações

In [13]:
sel.xpath("//table[3]//td[@class='nivel1']//span/text()").extract()[0]

'Oscilações'

In [14]:
oscilacoes_label = sel.xpath("//table[3]//td[@class='label w1']//span/text()").extract()
oscilacoes_label

['Dia',
 'Mês',
 '30 dias',
 '12 meses',
 '2021',
 '2020',
 '2019',
 '2018',
 '2017',
 '2016']

In [15]:
oscilacoes_dados = sel.xpath("//table[3]//span[@class='oscil']/font/text()").extract()
oscilacoes_dados

['-0,83%',
 '3,23%',
 '-17,53%',
 '20,08%',
 '-1,84%',
 '23,86%',
 '37,65%',
 '22,31%',
 '15,23%',
 '138,66%']

### Tabela 3.2: Indicadores fundamentalistas

In [16]:
sel.xpath("//table[3]//td[@class='nivel1']//span/text()").extract()[1]

'Indicadores fundamentalistas'

In [17]:
indicadores_label = sel.xpath("//table[3]//td[@class='label w2']//span[@class='txt']/text() | //table[3]//td[@class='label']//span[@class='txt']/text()").extract()
indicadores_label

['P/L',
 'LPA',
 'P/VP',
 'VPA',
 'P/EBIT',
 'Marg. Bruta',
 'PSR',
 'Marg. EBIT',
 'P/Ativos',
 'Marg. Líquida',
 'P/Cap. Giro',
 'EBIT / Ativo',
 'P/Ativ Circ Liq',
 'ROIC',
 'Div. Yield',
 'ROE',
 'EV / EBITDA',
 'Liquidez Corr',
 'EV / EBIT',
 'Div Br/ Patrim',
 'Cres. Rec (5a)',
 'Giro Ativos']

In [18]:
# Os dado estão vindo irregulares, com \n na frente de alguns valores
indicadores_dados = sel.xpath("//table[3]//td[@class='data w2']//span[@class='txt']/text() | //table[3]//td[@class='data']//span[@class='txt']/text()").extract()
indicadores_dados

['29,13',
 '0,82',
 '1,32',
 '18,18',
 '\n16,16',
 '\n9,9%',
 '\n1,04',
 '\n6,4%',
 '\n0,65',
 '\n3,6%',
 '\n3,18',
 '4,0%',
 '\n-4,21',
 '\n4,9%',
 '1,3%',
 '\n4,5%',
 '\n10,97',
 '\n2,41',
 '\n21,00',
 '\n0,62',
 '\n1,5%',
 '\n0,63']

In [19]:
# Limpando os valores, removendo os caracteres irregulares (\n)
nova_list = []
for valor in indicadores_dados:
    nova_list.append(valor.replace("\n", ""))
    
indicadores_dados = nova_list
indicadores_dados

['29,13',
 '0,82',
 '1,32',
 '18,18',
 '16,16',
 '9,9%',
 '1,04',
 '6,4%',
 '0,65',
 '3,6%',
 '3,18',
 '4,0%',
 '-4,21',
 '4,9%',
 '1,3%',
 '4,5%',
 '10,97',
 '2,41',
 '21,00',
 '0,62',
 '1,5%',
 '0,63']

### Tabela 4: Dados Balanço Patrimonial

In [20]:
sel.xpath("//table[4]//td[@class='nivel1']//span/text()").extract()[0]

'Dados Balanço Patrimonial'

In [21]:
balanco_patrimonial_label = sel.xpath("//table[4]//td[@class='label w2']//span[@class='txt']/text() | //table[4]//td[@class='label']//span[@class='txt']/text()").extract()
balanco_patrimonial_label

['Ativo',
 'Dív. Bruta',
 'Disponibilidades',
 'Dív. Líquida',
 'Ativo Circulante',
 'Patrim. Líq']

In [22]:
balanco_patrimonial_dados = sel.xpath("//table[4]//td[@class='data w3']//span[@class='txt']/text() | //table[4]//td[@class='data']//span[@class='txt']/text()").extract()
balanco_patrimonial_dados

['63.426.200.000',
 '19.541.000.000',
 '7.199.510.000',
 '12.341.500.000',
 '22.132.500.000',
 '31.270.600.000']

### Tabela 5.1: Demonstrativos de Resultados - Últimos 12 meses

In [23]:
sel.xpath("//table[5]//td[@class='nivel1']//span/text()").extract()[0]

'Dados demonstrativos de resultados'

In [24]:
sel.xpath("//table[5]//td[@class='nivel2 w5']//span/text()").extract()[0]

'Últimos 12 meses'

In [25]:
sel.xpath("//table[5]//td[@class='nivel2 w5']//span/text()").extract()[1]

'Últimos 3 meses'

In [26]:
demonstrativo_resultados_label = sel.xpath("//table[5]//td[@class='label w2']//span[@class='txt']/text() | //table[5]//td[@class='label']//span[@class='txt']/text()").extract()[0::2]
demonstrativo_resultados_label

['Receita Líquida', 'EBIT', 'Lucro Líquido']

In [27]:
demonstrativo_resultados_12_meses = sel.xpath("//table[5]//td[@class='data w3']//span[@class='txt']/text() | //table[5]//td[@class='data']//span[@class='txt']/text()").extract()[0::2]
demonstrativo_resultados_12_meses

['39.727.900.000', '2.553.280.000', '1.417.010.000']

In [28]:
demonstrativo_resultados_3_meses = sel.xpath("//table[5]//td[@class='data w3']//span[@class='txt']/text() | //table[5]//td[@class='data']//span[@class='txt']/text()").extract()[1::2]
demonstrativo_resultados_3_meses

['12.222.100.000', '1.326.630.000', '785.541.000']

# Acesso ao HDFS server (Write and Read)

In [57]:
empresa_label

['Papel',
 'Cotação',
 'Tipo',
 'Data últ cot',
 'Empresa',
 'Min 52 sem',
 'Setor',
 'Max 52 sem',
 'Subsetor',
 'Vol $ méd (2m)',
 'Valor de mercado',
 'Últ balanço processado',
 'Valor da firma',
 'Nro. Ações']

In [58]:
empresa_label = [
'Papel',
'Cotacao',
'Tipo',
'Data_ult_cot',
'Empresa',
'Min_52_sem',
'Setor',
'Max_52_sem',
'Subsetor',
'Vol_$_med_2m',
'Valor_de_mercado',
'Ult_balanço_processado',
'Valor_da_firma',
'Nro_Acoes']

In [59]:
# Dados precisam ser uma lista de tuples
empresa_dados_list = []
empresa_dados_list.append(tuple(empresa_dados))
empresa_dados_list

[('ITUB4',
  '26,08',
  'PN N1',
  '25/02/2021',
  'ITAUUNIBANCO PN N1',
  '20,36',
  'Intermediários Financeiros',
  '32,78',
  'Bancos',
  '1.160.530.000',
  '255.692.000.000',
  '31/12/2020',
  '-',
  '9.804.140.000')]

In [60]:
#!pip3 install pyspark

In [61]:
#from pyspark import SparkContext
#import sys

#if __name__ == "__main__":
#    sc = SparkContext(appName="RDD_Examples")
#    my_list = sc.parallelize(empresa_dados)

In [62]:
from pyspark.sql import SparkSession
sparkSession = SparkSession.builder.appName("example-pyspark-read-and-write").getOrCreate()

In [63]:
sparkSession.createDataFrame(empresa_dados_list).collect()

[Row(_1='ITUB4', _2='26,08', _3='PN N1', _4='25/02/2021', _5='ITAUUNIBANCO PN N1', _6='20,36', _7='Intermediários Financeiros', _8='32,78', _9='Bancos', _10='1.160.530.000', _11='255.692.000.000', _12='31/12/2020', _13='-', _14='9.804.140.000')]

In [64]:
sparkSession.createDataFrame(empresa_dados_list, empresa_label).collect()

[Row(Papel='ITUB4', Cotacao='26,08', Tipo='PN N1', Data_ult_cot='25/02/2021', Empresa='ITAUUNIBANCO PN N1', Min_52_sem='20,36', Setor='Intermediários Financeiros', Max_52_sem='32,78', Subsetor='Bancos', Vol_$_med_2m='1.160.530.000', Valor_de_mercado='255.692.000.000', Ult_balanço_processado='31/12/2020', Valor_da_firma='-', Nro_Acoes='9.804.140.000')]

In [65]:
df = sparkSession.createDataFrame(empresa_dados_list, empresa_label)
df

DataFrame[Papel: string, Cotacao: string, Tipo: string, Data_ult_cot: string, Empresa: string, Min_52_sem: string, Setor: string, Max_52_sem: string, Subsetor: string, Vol_$_med_2m: string, Valor_de_mercado: string, Ult_balanço_processado: string, Valor_da_firma: string, Nro_Acoes: string]

In [66]:
type(df)

pyspark.sql.dataframe.DataFrame

### Necessário configurar a versão do Python utilizada pelo Linux como a 3.6.
* sudo update-alternatives --install /usr/bin/python python /usr/bin/python3.4 1
* sudo update-alternatives --install /usr/bin/python python /usr/bin/python3.6 2
* sudo update-alternatives --config python
* sudo update-alternatives  --set python /usr/bin/python3.6

In [67]:
# Write into HDFS
# hdfs://hadoop-master:9000/user/hadoopuser/test/
df.write.save('hdfs://172.17.177.40:9000/user/hadoopuser/test/example', format='parquet', mode='append')

In [71]:
df_load = sparkSession.read.format('parquet').load('hdfs://172.17.177.40:9000/user/hadoopuser/test/example')

In [72]:
df_load.show()

+-----+-------+-----+------------+------------------+----------+--------------------+----------+--------------------+-------------+----------------+----------------------+---------------+--------------+
|Papel|Cotacao| Tipo|Data_ult_cot|           Empresa|Min_52_sem|               Setor|Max_52_sem|            Subsetor| Vol_$_med_2m|Valor_de_mercado|Ult_balanço_processado| Valor_da_firma|     Nro_Acoes|
+-----+-------+-----+------------+------------------+----------+--------------------+----------+--------------------+-------------+----------------+----------------------+---------------+--------------+
|PETR4|  23,19|   PN|  25/02/2021|      PETROBRAS PN|     11,29|Petróleo, Gás e B...|     31,12|Exploração, Refin...|2.449.750.000| 302.502.000.000|            31/12/2020|630.770.000.000|13.044.500.000|
|VALE3|  95,71|ON NM|  25/02/2021|        VALE ON NM|     32,82|           Mineração|    102,32|  Minerais Metálicos|2.973.250.000| 505.777.000.000|            30/09/2020|540.870.000.000| 

In [73]:
df_load.toPandas()

,Papel,Cotacao,Tipo,Data_ult_cot,Empresa,Min_52_sem,Setor,Max_52_sem,Subsetor,Vol_$_med_2m,Valor_de_mercado,Ult_balanço_processado,Valor_da_firma,Nro_Acoes
0,PETR4,"23,19",PN,25/02/2021,PETROBRAS PN,"11,29","Petróleo, Gás e Biocombustíveis","31,12","Exploração, Refino e Distribuição",2.449.750.000,302.502.000.000,31/12/2020,630.770.000.000,13.044.500.000
1,VALE3,"95,71",ON NM,25/02/2021,VALE ON NM,"32,82",Mineração,"102,32",Minerais Metálicos,2.973.250.000,505.777.000.000,30/09/2020,540.870.000.000,5.284.470.000
2,ITUB4,"26,08",PN N1,25/02/2021,ITAUUNIBANCO PN N1,"20,36",Intermediários Financeiros,"32,78",Bancos,1.160.530.000,255.692.000.000,31/12/2020,-,9.804.140.000
